## Semana 02 - Big Data

## Bibliotecas

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

## Setup

In [ ]:
import findspark
findspark.init()

In [ ]:
spark = SparkSession.builder.getOrCreate()

In [ ]:
df = spark.createDataFrame([('Pedro', '4'), ('João', '5')], schema = 'nome STRING, id STRING')

In [ ]:
df.show()

## Acessando os Tipos de Dados

In [ ]:
from pyspark.sql.types import *



In [ ]:
int_type = IntegerType()
int_type


In [ ]:
array_type = ArrayType(IntegerType())
array_type

## Convertendo os tipos de Colunas

In [ ]:
df.dtypes

In [ ]:
df.select('nome', col('id').cast(IntegerType())).dtypes

### forma alternativa
IntegerType() == 'int'
StringType() == 'string'
FloatType() == 'float'

ArrayType(IntegralType()) == 'ARRAY<INT>'

In [ ]:
df.select('nome', col('id').cast('int')).dtypes


## Schema e Criação de DataFrames
Um schema no Spark é uma especificação de tipos das colunas de um DataFrame. Eles são usados na leitura de dados externos e criação de DataFrames, e podem ser passados diretamente ao Spark ou podem ser inferidos. Passar um schema na leitura traz benefícios interessantes, como:

Evita que o Spark faça inferência de tipos, o que é custoso e demorado dependendo do tamanho do arquivo, além de propenso a erros;
Permite que o usuário identifique erros nos dados logo na leitura, caso os dados não sigam o schema especificado.

In [ ]:

df = spark.createDataFrame([('Pedro', 1), ('João', 6), ('Juliana', 4), ('Lucas', 7)], schema = ['nome', 'id'])

In [ ]:
df.show()

In [ ]:
df.dtypes

## Criando Schemas programaticamente

In [ ]:
schema = StructType([
    StructField('nome', StringType()),
    StructField('id', IntegerType())
])

In [ ]:
df = spark.createDataFrame([('Pedro', 1), ('João', 6), ('Juliana', 4), ('Lucas', 7)], schema = schema)

In [ ]:
df.dtypes

## Criando schemas com DDL

In [ ]:
schema = 'nome STRING, id INT'


In [ ]:
df = spark.createDataFrame([('Pedro', 1), ('João', 6), ('Juliana', 4), ('Lucas', 7)], schema = schema)

In [ ]:
df.dtypes

## Criando DataFrames

In [ ]:
data = [('Pedro', 1), ('João', 6), ('Juliana', 4), ('Lucas', 7)]

In [ ]:
schema = 'nome STRING, id INT'


In [ ]:
df = spark.createDataFrame(data, schema=schema)

In [ ]:
df.dtypes

In [ ]:
df.schema


In [ ]:
df.printSchema()

In [ ]:

spark.range(100).show()

In [ ]:
#df = spark.createDataFrame(pandas_df)

## Leitura e Escrita de Dados

### DataFrameReader
spark.read.format(format).option(args).load(file/path)

### DataFrameWriter
df.write.format(format).option(args).save(file/path)
Lendo e Escrevendo CSV
Opções mais comuns:

<li>header</li>
<li>inferSchema</li>
<li>sep</li>
<li>encoding</li>

In [ ]:
data_path = './'

In [ ]:
file_path = 'df_cnae.csv'

In [ ]:
df = spark.read.format('csv').load(file_path)

In [ ]:
df.limit(15).show()


## Definindo o schema¶

In [ ]:
schema = 'cod_cnae STRING, descricao_cnae STRING'


## Opção de Correção 1

In [ ]:
df = spark.read.csv(file_path, sep=';', encoding='ISO-8859-1', schema=schema)
df.limit(5).show()

## Opção de Correção 2¶

In [ ]:
df = (
    spark.read
    .format('csv')
    .option('sep', ';')
    .option('encoding', 'ISO-8859-1')
    .schema(schema)
    .load(file_path)
)
df.limit(5).show()

## Opção de Correção 3

In [ ]:
df = (
    spark.read
    .format('csv')
    .options(sep=';', encoding='ISO-8859-1')
    .schema(schema)
    .load(file_path)
)
df.limit(5).show()


Obs: utilizando o método "options" podemos parametrizar melhor nossa função usando um dicionário


In [ ]:
options_dict = {
    'sep': ';',
    'encoding': 'ISO-8859-1',
}

df = (
    spark.read
    .format('csv')
    .options(**options_dict)
    .schema('cod_cnae INT, descricao_cnae STRING')
    .load(file_path)
)
df.limit(5).show()

In [ ]:

df.printSchema()

In [ ]:
#df.write.format('csv').save('./' + 'df_cnaet_este', header=True)

In [ ]:
#spark.read.format('csv').load(data_path + 'df_cnaet_este', header=True).printSchema()


## Lendo e Escrevendo JSON

In [ ]:
df.write.format('json').save(data_path + 'df_cnae.json')

In [ ]:

df_json = spark.read.format('json').load(data_path + 'df_cnae.json')

In [ ]:
df_json.show()

In [ ]:
df_json.printSchema()

## Lendo e Escrevendo ORC

In [ ]:
formato = 'orc'
df.write.format(formato).save(data_path + 'df_cnae.' + formato)

In [ ]:
df_orc = spark.read.format(formato).load(data_path + 'df_cnae.orc')

In [ ]:
df_orc.printSchema()

## Lendo e Escrevendo Parquet
Armazenamento colunar, em contraste com o CSV, que armazena baseado nas linhas. Assim, quando uma query é realizada é possível ignorar os dados não relevantes de maneira rápida e fácil, resultando em operações bem mais eficientes;
Preservação de metadados, incluindo os tipos das colunas, o que garante eficiência e praticidade na escrita e leitura (não é necessário especificar schemas para arquivos parquet);
Suporte a dados estruturados de forma aninhada, como listas;
Otimizado para processar dados particionados com volume na casa dos gigabytes para cada arquivo;
Compressão de dados na escrita, de forma a ocupar menos espaço;
Integração com ferramentas como AWS Athena, Amazon Redshift Spectrum, Google BigQuery e Google Dataproc.


In [ ]:
df.write.format('parquet').save(data_path + 'df_cnae')

In [ ]:
df_parquet = spark.read.format('parquet').load(data_path + 'df_cnae')

In [ ]:
df_parquet.printSchema()


mode:

append: arquivos empilhados aos ja existentes
ignore: retorna um erro silencioso
overwrite: sobrescreve os dados já existente
error (default): retorne erro se já existem dados

In [ ]:
df.write.format('parquet').mode('overwrite').save(data_path + 'df_cnae')